Los resultados de este notebook nos demuestran como incluyendo ciertas metricas de feature selection, los resultados que podemos obtener, al menos con RF son iguales.

La razon que le atribuimos a esto es que RF tiene la capacidad para discriminar aquellas features que tienen menos capacidades predictivas, escogiendo aquellas mejores, por lo que , aunque pudiera ser una excelente manera para disminuir tiempo, ahora el tiempo del FS pudira complementar este ahorro.

In [55]:
import pandas as pd
import json
from copy import copy

In [33]:
with open("results.json" , "r") as fp:
    data = json.load( fp )

data.keys()

dict_keys(['AAL', 'AAPL', 'AAXJ', 'ABBV', 'ABT', 'AC', 'ACCELSAB', 'ACTINVRB', 'ACWI', 'AEROMEX', 'AGUA', 'ALEATIC', 'ALFAA', 'ALPEKA', 'ALSEA', 'AMD', 'AMXL', 'AMZN', 'ANB', 'ARA', 'ARISTOSA', 'ASURB', 'AUTLANB', 'AXP', 'AXTELCPO', 'AZTECACPO', 'BA', 'BABAN', 'BAC', 'BACHOCOB'])

In [34]:
methods = {}

for asset, v in data.items():
    
    for method, k in v["features"].items():
        
        if len(k) == 0: continue

        df = pd.DataFrame().from_dict( eval(k) )

        if method not in methods: 
            methods[ method ] = df
        else:

            methods[method] += df

In [35]:
target1 = pd.DataFrame()
for m, df in methods.items():

    if len(target1) == 0: target1 = df[["target_1"]]
    else: 
        target1 = pd.concat([ target1, df[["target_1"]] ], axis = 1)
    
    target1.rename(columns = {"target_1": m}, inplace = True)

target1

C:\Users\ramon\Trading\VirtualEnv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,causality,corr,mFCBF
reservas internacionales_1,0.0,0.0,2.0
s&p500_2,0.0,0.0,2.0
s&p500_4,0.0,0.0,4.0
wma_3_1,1.0,13.0,6.0
dema1,1.0,0.0,3.0
...,...,...,...
stoch_10,1.0,1.0,13.0
wma_6_4,0.0,0.0,11.0
ema_3_3,0.0,1.0,12.0
bb_20,1.0,0.0,2.0


In [53]:
target1.sum(axis = 1).sort_values(ascending=False).head(20)

ema_6_1        24.0
close_2        20.0
wma_3_1        20.0
dema_12_1      20.0
adj close_2    20.0
high_1         19.0
wma_6_1        18.0
dema_6_1       18.0
sma_3_1        17.0
cci_7          17.0
dema_3_2       17.0
stoch_6        16.0
wma_3_2        15.0
vpt            15.0
ema_3_1        15.0
adj close_3    15.0
cci_10         15.0
william_15     15.0
cci_4          15.0
stoch_12       15.0
dtype: float64

In [59]:
targets = target1.copy()
targets["causality"] *= 2
targets.sum(axis = 1).sort_values(ascending=False).head(25)

ema_6_1           25.0
wma_3_1           21.0
dema_12_1         21.0
close_2           20.0
cci_7             20.0
adj close_2       20.0
dema_6_1          19.0
high_1            19.0
wma_6_1           18.0
dema_3_2          18.0
sma_3_1           18.0
vpt               17.0
stoch_6           17.0
stoch_10          16.0
stoch_12          16.0
ema_3_1           16.0
william_15        16.0
cci_10            16.0
ema_3_2           16.0
wma_3_2           16.0
adj close_3       15.0
force_index_10    15.0
adj close_1       15.0
rsi_4             15.0
cci_4             15.0
dtype: float64

In [5]:
df[ df[ c ] > 0 ].index.values

array(['volume_3', 'william_15', 'tsi_10', 'vpt', 'desempleo_1',
       'volume_2', 'materiales', 'cetes', 'salud', 'materiales_1',
       'pib_1', 'consumo no basico_3', 'consumo no basico_4', 'desempleo',
       'volume_4', 'adj close_1', 'rsi_21', 'stoch_10', 'pib', 'rsi_14',
       'consumo frecuente_3', 'pib_4', 'consumo frecuente_4', 'volume',
       'mexbol_4', 'cetes_1', 'volume_5', 'adx_14'], dtype=object)

In [6]:
targets = {}

for asset, v in data.items():
    
    for target, k in v["regr"].items():

        for regr, j in k.items():

            if len(j) == 0: continue

            df = pd.DataFrame().from_dict(j) 
            df.set_index("param", inplace = True)
            df.rename( columns = {"error":asset}, inplace = True )

            if target not in targets: 
                targets[ target ] = df
                continue

            targets[ target ][asset] = df[asset].values


In [7]:
for t in targets:
    targets[t] = pd.DataFrame(targets[t].mean(axis = 1))
    targets[t].sort_values(by = 0, ascending = True)

In [8]:
targets["target_1"]

,0
param,
"{'criterion': 'absolute_error', 'n_estimators': 200}",0.001026
"{'criterion': 'squared_error', 'n_estimators': 200}",0.001102
"{'criterion': 'squared_error', 'n_estimators': 100}",0.001158
"{'criterion': 'absolute_error', 'n_estimators': 100}",0.001178
"{'criterion': 'absolute_error', 'n_estimators': 10}",0.001211
"{'criterion': 'squared_error', 'n_estimators': 20}",0.001245
"{'criterion': 'squared_error', 'n_estimators': 10}",0.001294
"{'criterion': 'squared_error', 'n_estimators': 50}",0.001380
"{'criterion': 'absolute_error', 'n_estimators': 20}",0.001777


In [67]:
with open("results_normal.json" , "r") as fp:
    ndata = json.load( fp )

ndata.keys()

dict_keys(['AAL', 'AAPL', 'AAXJ', 'ABBV', 'ABT', 'AC', 'ACCELSAB', 'ACTINVRB', 'ACWI', 'AEROMEX'])

In [73]:
targets = {}

for asset, v in ndata.items():
    
    for target, k in v["regr"].items():

        for regr, j in k.items():

            if len(j) == 0: continue

            df = pd.DataFrame().from_dict(j) 
            df.set_index("param", inplace = True)
            df.rename( columns = {"error":asset}, inplace = True )

            if target not in targets: 
                targets[ target ] = df
                continue

            targets[ target ][asset] = df[asset].values


In [74]:
for t in targets:
    targets[t] = pd.DataFrame(targets[t].mean(axis = 1))
    targets[t].sort_values(by = 0, ascending = True)

In [75]:
targets["target_1"]


,0
param,
"{'criterion': 'absolute_error', 'n_estimators': 100}",0.001027
"{'criterion': 'squared_error', 'n_estimators': 100}",0.001036
"{'criterion': 'absolute_error', 'n_estimators': 200}",0.001075
"{'criterion': 'squared_error', 'n_estimators': 200}",0.001120
"{'criterion': 'squared_error', 'n_estimators': 10}",0.001158
"{'criterion': 'absolute_error', 'n_estimators': 50}",0.001197
"{'criterion': 'absolute_error', 'n_estimators': 20}",0.001311
"{'criterion': 'squared_error', 'n_estimators': 50}",0.001403
"{'criterion': 'absolute_error', 'n_estimators': 10}",0.001537
